# Training A Multilayer Perceptron Using GPU P100

In [1]:
%%time

import numpy as np
import pandas as pd


from tensorflow.keras.models import load_model
from tensorflow.test import gpu_device_name

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

CPU times: user 4 s, sys: 1.2 s, total: 5.19 s
Wall time: 10.2 s


In [2]:
sample_sub = pd.read_csv('/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv', sep='\t')

In [3]:
%%time

# Check if GPU is available
if gpu_device_name():
    print('GPU device found: {}'.format(gpu_device_name()))
else:
    print("No GPU found. Training on CPU.")

No GPU found. Training on CPU.
CPU times: user 2.28 ms, sys: 1.22 ms, total: 3.5 ms
Wall time: 14.7 ms


In [4]:
%%time

train_data = np.load('/kaggle/input/t5embeds/train_embeds.npy')

with h5py.File('/kaggle/input/train-labels-cafa5/train_labels.h5', 'r') as hf:
    train_labels = hf['labels'][:]

In [5]:
%%time

test_data = np.load('/kaggle/input/t5embeds/test_embeds.npy')

CPU times: user 0 ns, sys: 1.21 s, total: 1.21 s
Wall time: 10.1 s


In [6]:
%%time

INPUT_SHAPE = [train_data.shape[1]]
BATCH_SIZE = int(train_data.shape[0] / 250)

model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(input_shape=INPUT_SHAPE),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=train_labels.shape[1],activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['binary_accuracy', tf.keras.metrics.AUC()]
)

In [7]:
%%time
model.fit(
        train_data, train_labels,
        batch_size=BATCH_SIZE,
        epochs=10
    )

In [8]:
model.save('MultiLayerPerceptron.h5')

In [9]:
%%time

model = load_model('/kaggle/input/train-labels-cafa5/MultiaLayerPerceptron.h5')

CPU times: user 284 ms, sys: 77.7 ms, total: 361 ms
Wall time: 596 ms


In [10]:
%%time

predictions =  model.predict(test_data)

4434/4434 [==============================] - 22s 5ms/step
CPU times: user 41.1 s, sys: 5.6 s, total: 46.7 s
Wall time: 26.3 s


In [11]:
labels = np.load('/kaggle/input/train-labels-cafa5/top_1500_labels.npy',allow_pickle=True)

In [12]:
test_protein_ids = np.load('/kaggle/input/t5embeds/test_ids.npy')
l = []
for k in list(test_protein_ids):
    l += [ k] * predictions.shape[1]

In [13]:
df_submission = pd.DataFrame(
    {
        'Protein ID': l,
        'GO Term ID': np.tile(labels, predictions.shape[0]),
        'Prediction': np.round(predictions.ravel(),3)
    }
)

In [14]:
df_submission.head()

,Protein ID,GO Term ID,Prediction
0,Q9CQV8,GO:0005575,0.745
1,Q9CQV8,GO:0008150,0.707
2,Q9CQV8,GO:0110165,0.718
3,Q9CQV8,GO:0003674,0.776
4,Q9CQV8,GO:0005622,0.591


In [15]:
df_submission.to_csv('submission.tsv', sep='\t', header=None, index= None)